In [5]:
import torch
import torch.nn as nn
from torchvision.models import resnet50

torch.random.manual_seed(1)

In [2]:
#Dataset and preprocessing

In [13]:
#Model
class ImageCNN(nn.Module):
    
    def __init__(self):
        super(ImageCNN, self).__init__()
        re = resnet50(pretrained=True)
        #remove the last classification layer
        self.resnet = nn.Sequential(*list(re.children())[:-1])
        self.fc = nn.Linear(2048, 2048)
        self.bn = nn.BatchNorm1d(2048)
        self.relu = nn.ReLU()
        self.dropOut = nn.Dropout(0.8)
        
    def forward(self, x):
        x = self.resnet(x) # (N,2048,1,1)
        x = torch.flatten(x,1) # (N,2048)
        x = self.relu(self.bn(self.fc(x)))
        x = self.relu(self.bn(self.fc(x))) # As per their MATLAB implementation
        return self.dropOut(x)

#Test Model -> Input (N,3,224,224) and Output (N,2048)
# net = ImageCNN()
# x= torch.rand((2,3,224,224))
# print(net(x).shape)    

torch.Size([2, 2048])

In [129]:
#textCNN -> The input should be the output of word2vec (N,300,32,1)
class BasicBlockText(nn.Module):
    
    def __init__(self, input_channel, intermediate_channel):
        super(BasicBlockText, self).__init__()
        self.bbConv1 = nn.Conv2d(input_channel, intermediate_channel, kernel_size=(1,1), stride=(1,1), padding=(0,0), bias=False)
        self.bbBatchNorm1 = nn.BatchNorm2d(intermediate_channel)
        self.relu = nn.ReLU()
        
        self.bbConv2 = nn.Conv2d(intermediate_channel, intermediate_channel, kernel_size=(1,2), stride=(1,1), padding=(1,1),bias=False)
        self.bbBatchNorm2 = nn.BatchNorm2d(intermediate_channel)
        
        self.bbConv3 = nn.Conv2d(intermediate_channel, input_channel, kernel_size=(1,1), stride=(1,1), padding=(0,0), bias=False)
        self.bbBatchNorm3 = nn.BatchNorm2d(input_channel)
        
    def forward(self, x):
        identity  = x
        out = self.relu(self.bbBatchNorm1(self.bbConv1(x)))
        out = self.relu(self.bbBatchNorm2(self.bbConv2(out)))
        out = self.bbBatchNorm3(self.bbConv3(out))
        
        out += identity
        return self.relu(out)

def __make__layer(input_channel, intermediate_channel, num_blocks):
    layers = []
    for i in range(num_block):
        layers.append(BasicBlockText(input_channel, intermediate_channel))
    return nn.Sequential(*layers)
        
class textCNN(nn.Module):
    
    def __init__(self, input_channel): #300
        super(textCNN, self).__init__()
        self.relu = nn.ReLU()
        #--------First CNN block----------------
        self.iConv1 = nn.Conv2d(input_channel, 128, 
                                 kernel_size=(1,1), stride=(1,1), 
                                 padding=(0,0), bias=False)
        self.b1bn1 = nn.BatchNorm2d(128)
        
        self.b1Conv2 = nn.Conv2d(128, 128, 
                                 kernel_size=(1,2), stride=(1,1), 
                                 padding=(0,1), bias=False, dilation=(1,2))
        self.b1bn2 = nn.BatchNorm2d(128)
        
        self.b1Conv3 = nn.Conv2d(128, 256, 
                                 kernel_size=(1,1), stride=(1,1), 
                                 padding=(0,0), bias=False)
        self.b1bn3 = nn.BatchNorm2d(256)
        
        #input here too
        self.iConv2 = nn.Conv2d(input_channel, 256,
                               kernel_size=(1,1), stride=(1,1), 
                               padding=(0,0), bias=False)
        self.b1bn4 = nn.BatchNorm2d(256)

    def forward(self, x):
        identity = x
        out = self.relu(self.b1bn1(self.iConv1(x)))
        out = self.relu(self.b1bn2(self.b1Conv2(out)))
        out = self.b1bn3(self.b1Conv3(out))
        print(out.shape)
        out2 = self.b1bn4(self.iConv2(identity))
        print(out2.shape)
        out2 +=out
        return self.relu(out2) # (N,256,32,1)

In [130]:
net = textCNN(300)

In [131]:
x = torch.rand(1,300,1,32)

In [132]:
y = net(x)

torch.Size([1, 256, 3, 32])
torch.Size([1, 256, 1, 32])


RuntimeError: The expanded size of the tensor (1) must match the existing size (3) at non-singleton dimension 2

In [128]:
y.shape

torch.Size([1, 256, 1, 32])